## Load in Required Packages

In [ ]:
from scipy.sparse import csr_matrix
from sqlalchemy import create_engine

import itertools
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import pymysql

# most_valuable_edge Functions for Girvin-Newman
from networkx.algorithms.centrality import edge_betweenness_centrality

# Clustering Algorithms
from networkx.algorithms.community.centrality import girvan_newman
from networkx.algorithms.community.quality import modularity
from networkx.algorithms.community.quality import performance

## Choose BERTopic or LDA
You'll need to run this notebook twice to generate the output needed for the UI. First, you'll need to run with BERTopic inputs. This will generate one set of network graph-related tables. Then, you'll need to run it for LDA, which produces a different set of tables.

You should utilize the variable in the next cell to make this selection. All other code should remain unchanged.

In [ ]:
# Select BERTopic or LDA in this cell
mode = "BERTopic"
# mode = "LDA"

In [ ]:
# Do not edit this cell or others below
if mode == "BERTopic":
    # Input
    source_doc_to_topic = "02a_bert_string_doc_to_topic"

    # Output
    topic_graph_table = "03_bert_topic_graph"
    paper_graph_table = "03_bert_paper_graph"
    topic_clust = "03_bert_topic_clust_output"
    topic_pagerank = "03_bert_topic_pagerank"
    paper_pagerank = "03_bert_paper_pagerank"
    topic_clust_csv = "03_bert_topic_clust_output.csv"
    topic_pagerank_csv = "03_bert_topic_pagerank.csv"
    paper_pagerank_csv = "03_bert_paper_pagerank.csv"


if mode == "LDA":
    # Input
    source_doc_to_topic = "02b_lda_string_doc_to_topic"

    # Output
    topic_graph_table = "03_lda_topic_graph"
    paper_graph_table = "03_lda_paper_graph"
    topic_clust = "03_lda_topic_clust_output"
    topic_pagerank = "03_lda_topic_pagerank"
    paper_pagerank = "03_lda_paper_pagerank"
    topic_clust_csv = "03_lda_topic_clust_output.csv"
    topic_pagerank_csv = "03_lda_topic_pagerank.csv"
    paper_pagerank_csv = "03_lda_paper_pagerank.csv"

## Read in the Needed Data

In [ ]:
# Configure MySQL Connection
sqlEngine = create_engine('mysql+pymysql://root:p@ssw0rd1@cse6242_team094_mysqldb/cse6242_team094')
dbConnection = sqlEngine.connect()

# Read in the topic output
df = pd.read_sql_table(source_doc_to_topic, con=dbConnection)

## Perform some preprocessing on topic output

In [ ]:
doc_id = np.array(df['cord_uid'])
doc_topic = np.array(df['topic'])

# Extract array of topic probabilities
topic_prob = []
for p in df['topic_prob']:
    row = [float(r) for r in p.strip('\n').replace(
        '[', '').replace(']', '').split()]
    topic_prob.append(row)

topic_prob = np.array(topic_prob)

# Array of Topic labels
topic_id = ['Topic_' + str(i) for i in range(topic_prob.shape[1])]

In [ ]:
# Convert arrays to dataframe
df_topic_prob = pd.DataFrame(
    data=topic_prob,
    index=np.array(doc_id),
    columns=topic_id)
df_topic_prob.insert(0, 'topic_id', doc_topic)

## Adjacency Matrix Functions

In [ ]:
# Probability threshold function
def probThreshold(data, threshold: float = 0.01):
    return np.where(data < threshold, 0, data)

In [ ]:
# Similarity measure
def simAbsCorr(data):
    S = np.absolute(np.corrcoef(data))
    return S


def simSignedCorr(data):
    S = (1 + np.corrcoef(data)) / 2
    return S

In [ ]:
# Adjacency functions
def powerAdj(SimMat, Beta: int = 6):
    A = SimMat ** Beta
    np.fill_diagonal(A, 0)
    return A


def signumAdj(SimMat, tau: float = 0.0):
    A = np.where(SimMat < tau, 0, 1)
    np.fill_diagonal(A, 0)
    return A

In [ ]:
# Topological Overlap Matrix function
def TOMadjacency(AdjMat, threshold_quantile: float = 0.8):
    '''
    TOMadjacency calculates an adjacency matrix by the network overlap of nodes
    in a weighted, undirected graph.
    '''
    # Calculate common neighbors of each node
    L = AdjMat.dot(AdjMat.T)

    # Calculate connectivity of node
    Krow = AdjMat.sum(axis=1)
    Kcol = AdjMat.sum(axis=0)
    Kmin = np.array([np.minimum(k_i, Kcol) for k_i in Krow])

    # Topological overlap
    TOM = (L + AdjMat) / (Kmin + 1 - AdjMat)

    TOM_filtered = np.where(
        TOM >= np.quantile(
            TOM, threshold_quantile), TOM, 0)

    np.fill_diagonal(TOM_filtered, 0)

    TOMlower = np.tril(TOM_filtered)

    TOMsparse = csr_matrix(TOMlower)

    return TOMsparse

## Filter Out Bad Data

In [ ]:
# Filter data to exclude small probabilites
thresh_val = 1 / topic_prob.shape[1]

topic_prob_sigProbs = probThreshold(topic_prob, threshold=thresh_val)

zeroTopic_doc = np.where(topic_prob_sigProbs.sum(axis=1) == 0)[0].tolist()

doc_kept = np.delete(doc_id, zeroTopic_doc)

topic_prob_filtered = np.delete(topic_prob_sigProbs, zeroTopic_doc, axis=0)

## Form and Save Topic Graph

This will produce a rather small graph. With the sample 5000 papers, the graph's edge list will be ~67KB as a CSV

In [ ]:
# Generate adjacency of topics
S_topic = simSignedCorr(topic_prob_filtered.T)

A_topic = signumAdj(S_topic, tau=np.quantile(S_topic, 0.8))

TOM_topic = TOMadjacency(A_topic, threshold_quantile=0.9)

# Create graph
topic_graph = nx.from_scipy_sparse_matrix(TOM_topic)

# Assign graph node names to topic id's
topic_label_mapping = dict(zip(topic_graph, topic_id))
topic_graph = nx.relabel_nodes(topic_graph, topic_label_mapping)

In [ ]:
# Convert the Edge List to Pandas
topic_graph_df = nx.to_pandas_edgelist(topic_graph)

# Insert the topic graph into MySQL
topic_graph_df.to_sql(topic_graph_table, con=dbConnection, if_exists='replace')

## Form and Save Paper Graph

This portion of the code is rather large and compute intensive. The resulting edge list, if saved as CSV, will be ~619MB.

In [ ]:
# Generate adjacency of papers
S_paper = simAbsCorr(topic_prob_filtered)

A_paper = powerAdj(S_paper, Beta=18)

TOM_paper = TOMadjacency(A_paper, threshold_quantile=0.9)

# Create graph
paper_graph = nx.from_scipy_sparse_matrix(TOM_paper)

# Assign graph node names to paper id's
paper_label_mapping = dict(zip(paper_graph, doc_kept))
paper_graph = nx.relabel_nodes(paper_graph, paper_label_mapping)

In [ ]:
# Convert the Edge List to Pandas
paper_graph_df = nx.to_pandas_edgelist(paper_graph)

# Insert the paper graph into MySQL
paper_graph_df.to_sql(paper_graph_table, con=dbConnection, if_exists='replace', chunksize=500)

## Define a Helper Class for Integrating with NetworkX

This class originally included additional methods, such as Kernighan Lin Bisection, Fluid Communities, and Greedy Modularity. However, these have been removed from the final code to keep only the needed portions.

In [ ]:
class UndirectedDocumentGraph():
    """
    This class will be used to form various graph representations of our document corpus.
    The graph representation can be created all at once or incrementally.
    """

    def __init__(self):
        # Create an empty, undirected NetworkX graph
        self.nx_graph = nx.Graph()

    ####################
    # Graph Formation
    ####################

    def merge_graph_from_sparse_scipy(self, sp_matrix):
        """
        Takes in a SciPy sparse matrix, representing our pair-wise document similarity, creates a new graph from
        it, then merges with any existing nodes.
        """
        # Load the new portion of the graph
        new_graph = nx.from_scipy_sparse_matrix(
            sp_matrix, parallel_edges=False, edge_attribute="weight")

        # An adjacency matrix will contain entries relating documents to themselves.
        # These should be removed from the graph
        new_graph.remove_edges_from(nx.selfloop_edges(new_graph))

        # Merge it with the existing representation
        self.nx_graph = nx.union(new_graph, self.nx_graph)

    ####################
    # Graph Similarity
    ####################
    def get_modularity(self, graph, communities):
        """
        Calculates the modularity of a given partition of a graph. This will be one number for the whole partitioning.
        NetworkX Doc:
        https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.quality.modularity.html#networkx.algorithms.community.quality.modularity
        """
        return modularity(graph, communities, weight="weight")

    def get_performance(self, graph, partition):
        """
        The performance of a partition is the ratio of the number of intra-community edges plus
        inter-community non-edges with the total number of potential edges
        NetworkX Doc:
        https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.quality.performance.html#networkx.algorithms.community.quality.performance
        """
        return performance(graph, partition)

    def girvan_newman(
            self,
            k: int,
            most_valuable_edge: str = "edge_betweenness_centrality"):
        """
        k - represents the number of tuples of communities from the algorithm
        most_valuable_edge - function used to get the edge removed at each iteration
        NetworkX Doc for the Girvan-Newman Method:
        https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.centrality.girvan_newman.html#networkx.algorithms.community.centrality.girvan_newman
        Helpful video explanation:
        https://youtu.be/LtQoPEKKRYM
        """
        if most_valuable_edge == "edge_betweenness_centrality_equal_weight":
            # Default option for Girvan-Newman, assumes all edges of weight 1
            comp = girvan_newman(self.nx_graph, edge_betweenness_centrality)
        elif most_valuable_edge == "edge_betweenness_centrality_equal_with_weight":
            # Take edge weight into account
            def get_edge(G):
                centrality = edge_betweenness_centrality(G, weight="weight")
                return max(centrality, key=centrality.get)
            print(get_edge(self.nx_graph))
            comp = girvan_newman(self.nx_graph, get_edge)
        elif most_valuable_edge == "least_similar":
            # Simple option of removing edge with least weight
            def get_edge(G):
                # Get edge based on the weight value (index 2 of triple)
                u, v, w = min(G.edges(data="weight"), key=itemgetter(2))
                return (u, v)
            comp = girvan_newman(self.nx_graph, get_edge)
        else:
            raise ValueError(
                "Invalid most_valuable_edge option for Girvan-Newman")

        # Create a list of dictionaries representing each row for the Pandas DF
        node_dict_list = []

        # Extract only the first specified number of communities and add them
        # to the dictionary
        num_communities = 2
        for communities in itertools.islice(comp, k):
            # Get the n number of communities
            community_tuple = tuple(sorted(c) for c in communities)

            # Calculate the modularity of the partitioning
            mod = self.get_modularity(self.nx_graph, community_tuple)

            # Calculate the performance of the partitioning
            perf = self.get_performance(self.nx_graph, community_tuple)

            # Loop through each of the communities
            for cluster_id in range(len(community_tuple)):
                # Get the list of nodes in the community
                nodes = community_tuple[cluster_id]

                # Loop through each of the nodes and form the Pandas DF row
                # dictionary
                for node in nodes:
                    row_dict = {
                        "algorithm": "Girvan-Newman",
                        "settings": "most_valuable_edge:" + most_valuable_edge,
                        "num_clusters": num_communities,
                        "cluster_id": cluster_id,
                        "node_id": node,
                        "modularity": mod,
                        "performance": perf
                    }
                    node_dict_list.append(row_dict)

            # Increment the community count
            num_communities += 1

        # Create a Pandas DF from the results
        clustering_df = pd.DataFrame(node_dict_list)

        return clustering_df

    def undirected_pagerank(self, alpha: float = 0.85, max_iter: int = 100):
        """
        NetworkX Doc:
        https://networkx.org/documentation/stable//reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html
        """
        # Get the dictionary output from the pagerank algorithm
        pagerank_output = nx.pagerank(
            self.nx_graph,
            weight="weight",
            alpha=alpha,
            max_iter=max_iter)

        # Change the output to a Pandas DataFrame
        node_id_list = []
        page_rank_val_list = []
        for node_id, page_rank_val in pagerank_output.items():
            node_id_list.append(node_id)
            page_rank_val_list.append(page_rank_val)

        page_rank_df = pd.DataFrame.from_dict(data={
            "node_id": node_id_list,
            "page_rank_val": page_rank_val_list
        })

        # Sort by page rank value, descending
        page_rank_df.sort_values(
            by=['page_rank_val'],
            ascending=False,
            inplace=True)

        return page_rank_df

## Utilize the Helper Class for Community Detection

In [ ]:
# Create topic graph using helper class
topic_graph = UndirectedDocumentGraph()

topic_graph.merge_graph_from_sparse_scipy(TOM_topic)

In [ ]:
# Create the paper graph using helper class
paper_graph = UndirectedDocumentGraph()

paper_graph.merge_graph_from_sparse_scipy(TOM_paper)

In [ ]:
print("Girvan-Newman (edge_betweenness_centrality_equal_with_weight):")

girvan_newman_df_ebcw = topic_graph.girvan_newman(20, "edge_betweenness_centrality_equal_with_weight")

girvan_newman_df_ebcw_diag = girvan_newman_df_ebcw[[
    'num_clusters', 'modularity', 'performance']].drop_duplicates(ignore_index=True)

print(girvan_newman_df_ebcw_diag, "\n")

In [ ]:
# Save the clustering of topics to MySQL
girvan_newman_df_ebcw.iloc[:, 2:].to_sql(topic_clust, con=dbConnection, if_exists='replace')

In [ ]:
# Save the clustering of topics to CSV
output_path = topic_clust_csv
girvan_newman_df_ebcw.iloc[:, 2:].to_csv(output_path, sep="|", header=True, index=False)

In [ ]:
# Get graph page ranks
topic_pagerank_df = topic_graph.undirected_pagerank()
paper_pagerank_df = paper_graph.undirected_pagerank()

In [ ]:
# Save pagerank lists to MySQL
topic_pagerank_df.to_sql(topic_pagerank, con=dbConnection, if_exists='replace')

paper_pagerank_df.to_sql(paper_pagerank, con=dbConnection, if_exists='replace')

In [ ]:
# Save pagerank lists to CSV
topic_pagerank_df.to_csv(topic_pagerank_csv, sep="|", header=True, index=False)

paper_pagerank_df.to_csv(paper_pagerank_csv, sep="|", header=True, index=False)

In [ ]:
# Diagnostics plot, modularity and performance

fig, (ax1, ax2) = plt.subplots(2, 1)
fig.suptitle(
    'Girvan-Newman (edge_betweenness_centrality_equal_with_weight): 5k sample')
ax1.plot(
    girvan_newman_df_ebcw_diag['num_clusters'],
    girvan_newman_df_ebcw_diag['modularity'],
    '.-')
ax1.axvline(x=6, color='red')
ax1.set_ylabel('Modularity')
ax2.plot(
    girvan_newman_df_ebcw_diag['num_clusters'],
    girvan_newman_df_ebcw_diag['performance'],
    '.-',
    color='green')
ax2.axvline(x=6, color='red')
ax2.set_ylabel('Performance')
ax2.set_xlabel('# Clusters')
plt.show()